# Capstone Project for IBM Data Science Certificate

## Segmenting and Clustering Neighborhoods in Toronto

In [38]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [39]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
print(df[0].shape)
tor_data = df[0]
tor_data = tor_data[tor_dara['Borough']!='Not assigned']
print(tor_data.shape)

(180, 3)
(103, 3)


In [40]:
tor_data.reset_index(inplace = True)
tor_data.drop(['index'], axis = 1,inplace = True)


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [41]:
tor_data.shape

(103, 3)

In [42]:
tor_data.head()


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [43]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
tor_data = tor_data.merge(geo_data, how = 'left',left_on = 'Postal code', right_on = 'Postal Code')
tor_data.drop(['Postal Code'], axis = 1, inplace = True)
tor_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


## Define Foursquare Info

In [45]:
CLIENT_ID = 'Y4DKOYK4CQ2OYJNNVPJA5YJLQUOGBA0W1SVMFQBED2R5ZBK3' # your Foursquare ID
CLIENT_SECRET = 'LHYPOEWCPURSACTYWPUB02HTUURYGBXLMURMD42ZTIOLQYSR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y4DKOYK4CQ2OYJNNVPJA5YJLQUOGBA0W1SVMFQBED2R5ZBK3
CLIENT_SECRET:LHYPOEWCPURSACTYWPUB02HTUURYGBXLMURMD42ZTIOLQYSR


Get the Top 50 Venues in Postal Code M5A with a Radius of 500

In [46]:
dt_tor_lat = tor_data.loc[tor_data['Postal code']=='M5A',:]['Latitude']
dt_tor_long = tor_data.loc[tor_data['Postal code']=='M5A','Longitude']
dt_tor_neighborhood_name = tor_data.loc[tor_data['Postal code']=='M5A','Neighborhood']




In [26]:
radius = 500
LIMIT = 50
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, 43.65426, -79.360636, VERSION, radius, LIMIT)

In [27]:
results = requests.get(url).json()

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [48]:
results['response']['venues']

[{'id': '5bdc6c2bba57b4002c4c71a8',
  'name': 'Oldtown Bodega',
  'location': {'lat': 43.653966,
   'lng': -79.360752,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.653966,
     'lng': -79.360752}],
   'distance': 34,
   'postalCode': 'M5A 1L6',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['Toronto ON M5A 1L6', 'Canada']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1586744261',
  'hasPerk': False},
 {'id': '4bc70f5d14d7952126a066e9',
  'name': 'Sackville Playground',
  'location': {'address': '420 king st E',
   'lat': 43.65465604258614,
   'lng': -79.35987064632435,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.65465604258614,
     'lng': -79.35987064632435}],
   'distance': 75,
   'cc

In [51]:
from pandas.io.json import json_normalize
venues = json_normalize(results['response']['venues'])

In [54]:
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues = venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Oldtown Bodega,Café,43.653966,-79.360752
1,Sackville Playground,Park,43.654656,-79.359871
2,Body Blitz Spa East,Spa,43.654735,-79.359874
3,Tandem Coffee,Coffee Shop,43.653559,-79.361809
4,Starbucks,Coffee Shop,43.653844,-79.361283


In [58]:
unique_cat = nearby_venues['categories'].nunique()

In [61]:
print('There are {} unique categories of veunes around postal code M5A.'.format(unique_cat))

There are 38 unique categories of veunes around postal code M5A.
